<div style='float: right'><img src='pic/suiri.png'/></div>
## <div id='suiri' />推理パズル

In [ ]:
import numpy as np
from itertools import product
from pulp import *
from ortoolpy import addvar, addvars, addbinvar, addbinvars
kinds = [['akira','isamu','tadashi','hiroshi'],
         ['umbrella','shoes','paper','string'],
         ['red','blue','white','black']]
dic = {k:v for kind in kinds for v, k in enumerate(kind)}
data = [s.split(',') for s in """\
1,akira,,white
0,akira,string,
1,,paper,blue
0,isamu,paper,
1,tadashi,shoes,
0,tadashi,,red""".split()]

### 問題
* 3つの組(kinds)を入力し、各組間の対応を求めます
* ヒント(data)の意味
    * 明は白いのを買った
    * 明は糸じゃない
    * 青い紙を買った人がいる
    * 勇は紙じゃない
    * 正は靴を買った
    * 正は赤じゃない

### 変数
* $v_{ijk}$: 表iの位置(j,k) (1)

### 制約
* 縦横で丸は1つ (2)
* AかつB、BかつCなら、CかつA (3)
* ヒントを満たすこと (4)

In [ ]:
m = LpProblem()
v = np.array(addbinvars(3, 4, 4)) # (1)
for i in range(4):
    for j in range(3):
        m += lpSum(v[j,i,:]) == 1 # (2)
        m += lpSum(v[j,:,i]) == 1 # (2)
    for j, k, l in product(range(4), range(4), range(3)):
        m += v[l,i,j] + v[(l+1)%3,j,k] - v[(l+2)%3,k,i] <= 1 # (3)
for h in data:
    c0, c1, c2, c3 = int(h[0]), *[dic.get(i) for i in h[1:]]
    for i, (j, k) in enumerate([(c1,c2), (c2,c3), (c3,c1)]):
        if None not in (j, k):
            m += v[i,j,k] == c0 # (6)
%time m.solve()
r = np.vectorize(value)(v)
for i in range(4):
    j = r[0][i].argmax()
    print(kinds[0][i], kinds[1][j], kinds[2][r[1][j].argmax()])